In [1]:
import torch
s=5
torch.tril(torch.ones(size=(s,s))).type(torch.bool)

tensor([[ True, False, False, False, False],
        [ True,  True, False, False, False],
        [ True,  True,  True, False, False],
        [ True,  True,  True,  True, False],
        [ True,  True,  True,  True,  True]])

In [2]:
attention_mask = torch.Tensor([
         [0., 0., 1., 1., 1.],
         [1., 1., 1., 1., 1.]
        ])

torch.sum(attention_mask,dim=1)

tensor([3., 5.])

In [3]:
tmp_attn=torch.sum(attention_mask,dim=1,dtype=torch.uint8)
mask=torch.zeros(size=(attention_mask.shape[0],1,s,s),dtype=torch.bool)
for b in range(attention_mask.shape[0]):
    mask[b,0,s-tmp_attn[b]:,s-tmp_attn[b]:]=torch.tril(torch.ones(size=(tmp_attn[b],tmp_attn[b])))

mask

tensor([[[[False, False, False, False, False],
          [False, False, False, False, False],
          [False, False,  True, False, False],
          [False, False,  True,  True, False],
          [False, False,  True,  True,  True]]],


        [[[ True, False, False, False, False],
          [ True,  True, False, False, False],
          [ True,  True,  True, False, False],
          [ True,  True,  True,  True, False],
          [ True,  True,  True,  True,  True]]]])

In [1]:
# mha = MultiHeadAttention(d, h, 0.0)
b, s, d, h, hd = 16, 128, 768, 12, 64
from hw1.src.model import MultiHeadAttention
from einops import rearrange
import torch.nn.functional as F

mha = MultiHeadAttention(d, h, 0.0)
x = torch.rand(b, s, d)
q, kT, v = mha.q_kT_v(x)
k = rearrange(kT, "b h hd s -> b h s hd")

attn = mha.self_attention(q, kT, v)
attn_ref_multihead = F.scaled_dot_product_attention(q, k, v, is_causal=True)
attn_ref = rearrange(attn_ref_multihead, "b h s hd -> b s (h hd)")

assert torch.allclose(attn, attn_ref, atol=1e-5, rtol=1e-3)

x = torch.rand(2, 5, d)
q, kT, v = mha.q_kT_v(x)
k = rearrange(kT, "b h hd s -> b h s hd")
attention_mask = torch.tensor(
        [[0.0, 0.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0, 1.0]]
    )
attention_mask_with_causal = torch.tensor(
        [
            [
                [False, False, False, False, False],
                [False, False, False, False, False],
                [False, False, True, False, False],
                [False, False, True, True, False],
                [False, False, True, True, True],
            ],
            [
                [True, False, False, False, False],
                [True, True, False, False, False],
                [True, True, True, False, False],
                [True, True, True, True, False],
                [True, True, True, True, True],
            ],
        ]
    )[:, None]
attn = mha.self_attention(q, kT, v, attention_mask=attention_mask)
attn_ref_multihead = F.scaled_dot_product_attention(
        q, k, v, attn_mask=attention_mask_with_causal
    )
attn_ref = rearrange(attn_ref_multihead, "b h s hd -> b s (h hd)")


ModuleNotFoundError: No module named 'utils'

In [5]:
import torch
import torch.nn as nn
position_ids = torch.Tensor([
         [0, 0, 0, 1, 2],
         [0, 1, 2, 3, 4]
        ]).type(torch.long)

class a(nn.Module):
        def __init__(self):
                super().__init__()
                self.position_embeddings = nn.Embedding(5, 10)
        def forward(self,x):
                return self.position_embeddings(x)
        
aa=a()
aa(position_ids).shape


torch.Size([2, 5, 10])

In [6]:
position_ids = torch.arange(5).unsqueeze(0).expand(12, -1)

In [7]:
position_ids

tensor([[0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4]])